In [1]:
from function import *

/home/wmbio/anaconda3/envs/deepdep_paper/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wmbio/anaconda3/envs/deepdep_paper/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wmbio/anaconda3/envs/deepdep_paper/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wmbio/anaconda3/envs/d

In [14]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/prediction")
print(os.getcwd())

/home/wmbio/WORK/gitworking/Dependency_prediction
/home/wmbio/WORK/gitworking/Dependency_prediction/prediction


* **Path**

In [83]:
MODEL_PATH = "DepOI_1298_model/2022-0802/"
MODEL_NATM = "model_mut_exp"
PREDICT_PATH = "../preprocessing/PREDICTION/2022-08-03/"
BARCODE = "exp-mut-cna-methpredict_wmbio_ccls_"

* **Model load**

In [81]:
with tf.device('/cpu:0'):
    model_saved = models.load_model(MODEL_PATH + "%s.h5" % MODEL_NATM)

* **Predict Set load**

In [23]:
"""
exp..mut..cna..meth
exp..mut..cna
exp..mut

mut..cna..meth
"""

# load TCGA genomics data and gene fingerprints
data_mut, data_labels_mut, sample_names_mut, gene_names_mut = load_data_prediction(PREDICT_PATH + BARCODE + "mut_prediction.txt")
data_exp, data_labels_exp, sample_names_exp, gene_names_exp = load_data_prediction(PREDICT_PATH + BARCODE + "exp_prediction.txt")
data_cna, data_labels_cna, sample_names_cna, gene_names_cna = load_data_prediction(PREDICT_PATH + BARCODE + "cna_prediction.txt")
data_meth, data_labels_meth, sample_names_meth, gene_names_meth = load_data_prediction(PREDICT_PATH + BARCODE + "meth_prediction.txt")
data_fprint_DepOIs, data_labels_fprint, gene_names_fprint, function_names_fprint = load_data_prediction(PREDICT_PATH + BARCODE + "fingerprint_prediction.txt")
print("\n\nDatasets successfully loaded.\n\n")

* **Prediction**

In [77]:
batch_size = 200
with tf.device('/cpu:0'):
    t = time.time()
    data_pred = np.zeros((data_exp.shape[0], data_fprint_DepOIs.shape[0]))
    for z in np.arange(0, data_exp.shape[0]):
        data_pred_tmp = model_saved.predict([
            data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)
        data_pred[z] = np.transpose(data_pred_tmp)
        print("Unscreended sample %d predicted..." % z)

Unscreended sample 0 predicted...
Unscreended sample 1 predicted...
Unscreended sample 2 predicted...
Unscreended sample 3 predicted...
Unscreended sample 4 predicted...
Unscreended sample 5 predicted...
Unscreended sample 6 predicted...
Unscreended sample 7 predicted...
Unscreended sample 8 predicted...
Unscreended sample 9 predicted...
Unscreended sample 10 predicted...
Unscreended sample 11 predicted...
Unscreended sample 12 predicted...
Unscreended sample 13 predicted...
Unscreended sample 14 predicted...
Unscreended sample 15 predicted...
Unscreended sample 16 predicted...
Unscreended sample 17 predicted...
Unscreended sample 18 predicted...
Unscreended sample 19 predicted...
Unscreended sample 20 predicted...
Unscreended sample 21 predicted...
Unscreended sample 22 predicted...
Unscreended sample 23 predicted...
Unscreended sample 24 predicted...
Unscreended sample 25 predicted...
Unscreended sample 26 predicted...


In [78]:
# write prediction results to txt
data_pred_df = pd.DataFrame(data=np.transpose(data_pred), index=gene_names_fprint, columns=sample_names_mut[0:data_exp.shape[0]])
data_pred_df.to_csv('custom_mut_exp.csv')